In [2]:
import requests
import pandas as pd
import time
import glob
import os

In [2]:
file_path = "steam_ids.csv"
df = pd.read_csv(file_path, header=None, names=['steamid'])
df.to_csv(file_path, index=False)  

In [5]:
# Ganti dengan Steam API Key Anda
API_KEY = '82FE7EFE9A6F07C9D4EA7F7D1E8F71A4'

# Fungsi untuk mengambil nama game berdasarkan AppID
def get_game_name(appid):
    print(f"📌 Mengambil nama game untuk AppID: {appid}")
    url = f'https://store.steampowered.com/api/appdetails?appids={appid}&cc=us'
    response = requests.get(url)

    if response.status_code != 200:
        print(f"⚠️ Gagal mengambil nama game untuk AppID: {appid}")
        return 'Unknown'

    try:
        data = response.json()
        if data.get(str(appid), {}).get('success', False):
            return data[str(appid)]['data'].get('name', 'Unknown')
    except ValueError:
        return 'Unknown'

    return 'Unknown'

# Fungsi untuk mendapatkan genre game
def get_game_genre(appid):
    print(f"📌 Mengambil genre untuk AppID: {appid}")
    url = f'https://store.steampowered.com/api/appdetails?appids={appid}&cc=us'
    response = requests.get(url)

    if response.status_code != 200:
        print(f"⚠️ Gagal mengambil genre untuk AppID: {appid}")
        return []

    try:
        data = response.json()
        if data.get(str(appid), {}).get('success', False):
            genres = data[str(appid)]['data'].get('genres', [])
            return [genre['description'] for genre in genres]
    except ValueError:
        return []

    return []

# Fungsi untuk mengambil jumlah achievement yang sudah dicapai pemain
def get_game_achievements(steam_id, appid):
    print(f"🏆 Mengambil achievement untuk Steam ID: {steam_id}, AppID: {appid}")
    url = f'https://api.steampowered.com/ISteamUserStats/GetPlayerAchievements/v1/?key={API_KEY}&steamid={steam_id}&appid={appid}'
    response = requests.get(url)

    if response.status_code != 200:
        print(f"⚠️ Gagal mengambil achievement untuk Steam ID: {steam_id}, AppID: {appid}")
        return 0

    try:
        data = response.json()
        if 'playerstats' in data and 'achievements' in data['playerstats']:
            return sum(1 for ach in data['playerstats']['achievements'] if ach.get('achieved') == 1)
    except ValueError:
        return 0

    return 0

# # Fungsi untuk mengambil review game
# def get_game_reviews(appid, num_reviews=5):
#     print(f"📝 Mengambil review untuk AppID: {appid}")
#     url = f'https://store.steampowered.com/appreviews/{appid}?json=1&language=all'
#     response = requests.get(url)

#     if response.status_code != 200:
#         print(f"⚠️ Gagal mengambil review untuk AppID: {appid}")
#         return []

#     try:
#         data = response.json()
#         return [review['review'] for review in data.get('reviews', [])[:num_reviews]]
#     except ValueError:
#         return []

# Fungsi untuk mengambil daftar game dari Steam ID
def get_owned_games_by_steam_id(steam_id, index, total):
    print(f"\n🔄 [{index+1}/{total}] Mengambil daftar game untuk Steam ID: {steam_id}")
    url = f'https://api.steampowered.com/IPlayerService/GetOwnedGames/v1/?key={API_KEY}&steamid={steam_id}&format=json'
    response = requests.get(url)

    if response.status_code != 200:
        print(f"⚠️ Gagal mengambil daftar game untuk Steam ID: {steam_id}")
        return []

    try:
        data = response.json()
        games_data = []
        games = data.get('response', {}).get('games', [])
        print(f"✅ Steam ID {steam_id} memiliki {len(games)} game.")

        for i, game in enumerate(games):
            appid = game['appid']
            playtime = game['playtime_forever'] / 60
            genres = get_game_genre(appid)
            game_name = get_game_name(appid)
            achievement_count = get_game_achievements(steam_id, appid)

            games_data.append({
                'Steam ID': steam_id,
                'App ID': appid,
                'Game Name': game_name,
                'Playtime (hours)': playtime,
                'Genres': ', '.join(genres),
                'Achievements': achievement_count
            })

            print(f"   🕹️ [{i+1}/{len(games)}] {game_name} berhasil diproses! 🎮")

            time.sleep(2)  # Hindari rate limiting

        return games_data
    except ValueError:
        return []

# Fungsi untuk mengambil Steam IDs dari file CSV
def get_steam_ids_from_csv(file_path):
    df = pd.read_csv(file_path)
    return df['steamid'].astype(str).tolist()

# Fungsi utama untuk mengambil data dari beberapa Steam ID dalam satu file CSV
def collect_steam_data(steam_ids_file, output_file):
    print(f"\n📂 Memproses file: {steam_ids_file}")

    all_games_data = []
    steam_ids = get_steam_ids_from_csv(steam_ids_file)
    total_ids = len(steam_ids)
    
    print(f"🔢 Ditemukan {total_ids} Steam ID dalam file.")

    for idx, steam_id in enumerate(steam_ids):
        user_games = get_owned_games_by_steam_id(steam_id, idx, total_ids)
        all_games_data.extend(user_games)

    if all_games_data:
        df = pd.DataFrame(all_games_data)
        df.to_csv(output_file, index=False, encoding='utf-8')
        print(f"\n✅ Data dari {steam_ids_file} berhasil disimpan dalam {output_file}")
    else:
        print(f"⚠️ Tidak ada data yang dapat disimpan dari {steam_ids_file}.")

# **Menggunakan glob untuk membaca semua file dalam folder**
folder_path = "fixing_id"  # Folder tempat file CSV berada
output_folder = "finished_fixing_id"  # Folder untuk menyimpan hasil scraping

# Buat folder output jika belum ada
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Cari semua file CSV dalam folder yang ditentukan
file_list = glob.glob(os.path.join(folder_path, "*.csv"))

# Urutkan file berdasarkan angka dalam namanya
file_list = sorted(file_list, key=lambda x: int(os.path.basename(x).split('_')[-1].split('.')[0]))


# Loop melalui semua file yang ditemukan dan proses satu per satu
for file_path in file_list:
    file_name = os.path.basename(file_path)  # Ambil nama file saja
    output_file = os.path.join(output_folder, f"processed_{file_name}")  # Simpan dengan nama baru
    collect_steam_data(file_path, output_file)

print("\n🎉 Semua file telah diproses!")



📂 Memproses file: fixing_id/steamid_asing_1.csv
🔢 Ditemukan 57 Steam ID dalam file.

🔄 [1/57] Mengambil daftar game untuk Steam ID: 76561198184561219
✅ Steam ID 76561198184561219 memiliki 100 game.
📌 Mengambil genre untuk AppID: 240
📌 Mengambil nama game untuk AppID: 240
🏆 Mengambil achievement untuk Steam ID: 76561198184561219, AppID: 240
   🕹️ [1/100] Counter-Strike: Source berhasil diproses! 🎮
📌 Mengambil genre untuk AppID: 4000
📌 Mengambil nama game untuk AppID: 4000
🏆 Mengambil achievement untuk Steam ID: 76561198184561219, AppID: 4000
   🕹️ [2/100] Garry's Mod berhasil diproses! 🎮
📌 Mengambil genre untuk AppID: 3480
📌 Mengambil nama game untuk AppID: 3480
🏆 Mengambil achievement untuk Steam ID: 76561198184561219, AppID: 3480
⚠️ Gagal mengambil achievement untuk Steam ID: 76561198184561219, AppID: 3480
   🕹️ [3/100] Peggle Deluxe berhasil diproses! 🎮
📌 Mengambil genre untuk AppID: 220
📌 Mengambil nama game untuk AppID: 220
🏆 Mengambil achievement untuk Steam ID: 76561198184561219

KeyboardInterrupt: 